# Análisis de Datos de Tiros en Baloncesto: Mejorando el Rendimiento de Jugadores Universitarios

### Objetivos

- 🗺️ Explorar: Identificar la distancia desde el aro en la que cada jugador tiene mayor probabilidad de anotar un tiro, utilizando los datos disponibles.
- 📊 Visualizar: Generar gráficos que muestren las posiciones de los tiros en la cancha, diferenciando entre los resultados (acierto/fallo) y los cuatro jugadores analizados.
- 🔎 Analizar: Evaluar si existe una relación directa entre la proximidad al aro y la probabilidad de anotar un tiro, basada en los patrones identificados en los datos.

---

### Contexto

Un equipo de baloncesto universitario busca aprovechar el análisis de datos para optimizar el desempeño de sus jugadores. Como proyecto inicial, se han proporcionado datos históricos de tiros en la NBA correspondientes a cuatro jugadores específicos. El objetivo es determinar si estos datos pueden ofrecer recomendaciones personalizadas sobre las áreas de mayor efectividad en la cancha para cada jugador.

El entrenador plantea una pregunta clave:
> ¿Es posible generar estrategias basadas en datos para aumentar la probabilidad de éxito en los tiros de cada jugador?

---

## Metodologia

### Datos

El *dataset* esta disponible [aqui](https://kaggle.com/datasets/163acec12e8bdb031c78f4912406ee3e68e3a423529469b4d07a0637c0f96bfc). Contamos con una cantidad de **776** registros para su analisis acerca de los jugadores **Chris Paul, Russell Westbrook, Seth Curry,Trae Young**.El dataset utilizado está disponible aquí e incluye 776 registros correspondientes a los siguientes jugadores:

- Chris Paul
- Russell Westbrook
- Seth Curry
- Trae Young

### Diccionario de datos

|variable|clase|descripción|
|---|---|---|
|SHOOTER|String|Nombre del jugador que realiza el tiro|
|X|float|Distancia horizontal del tiro realizado desde el aro en pies|
|Y|float|Distancia vertical del tiro realizado desde el aro en pies|
|RANGE|String|Rango de radio del tiro realizado desde el aro en pies|
|DEFENDER|String|Nombre del jugador que defiende el tiro|
|SCORE|String|'MADE' si el tiro es anotado, de lo contrario 'MISSED'|


### Procesamiento

Un análisis inicial reveló que el conjunto de datos no presentaba valores faltantes ni inconsistencias. Por lo tanto, no fue necesario realizar una limpieza extensiva. Los pasos clave incluyeron:

1. **Revisión exploratoria**: Inspección de la distribución de las variables y detección de posibles anomalías.
2. **Verificación de calidad**: Confirmación de que los datos eran consistentes y adecuados para el análisis.

### Tools 

- Python: Versión 3.12
- Conda: 24.9.2 (gestión de entornos)
- Plotly: 5.24.1 (visualización interactiva)
- Scipy: 1.14.1 (análisis estadístico)
- Pandas: 2.2.2 (procesamiento y manipulación de datos)
- Numpy: 1.26.4 (cálculos numéricos)



In [104]:
# LIBS 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm
import pandas as pd

nba = pd.read_csv('college_nba_players_shooting.csv', index_col=0)

### Analisis Exploratorio (EDA)


#### Grafico de dispersion
Representación de tiros en la cancha, diferenciando entre los cuatro jugadores.

In [85]:
fig = px.scatter(
    data_frame=nba,
    x='Y',
    y='X',
    color='SHOOTER',
    hover_data=['SCORE'],
    )

fig.show()

#### Mapas de Calor
Identificación de áreas de la cancha donde los jugadores tienen más probabilidades de anotar.

In [98]:
total_attempts = nba.groupby(['SHOOTER','RANGE']).size().reset_index(name='TOTAL_ATTEMPTS')

total_success = nba[nba['SCORE'] == 'MADE'].groupby(['SHOOTER','RANGE']).size().reset_index(name='SUCCESS_ATTEMPTS')

probabilities = total_attempts.merge(total_success)


probabilities.loc[:, 'PROBABILITY'] = probabilities.apply(
    lambda row: round(row['SUCCESS_ATTEMPTS'] / row['TOTAL_ATTEMPTS'], 4),
    axis=1
)

fig = px.imshow(
    probabilities.pivot(index='SHOOTER', columns='RANGE', values='PROBABILITY'),
    labels=dict(x="Rango", y="Jugador", color="Probability"),
    title="Probabilidad de anotar por Jugador y Rango",
    color_continuous_scale="RdBu_r",
    aspect="auto",
    text_auto=True
)


fig.show()

#### Interpretacion

**Generales**:

- La mayoría de los jugadores, **excepto Russell Westbrook**, tienen una buena probabilidad de anotar en el rango cercano al aro (0 a 4 pies).
- Esto confirma que cuanto más cerca están del aro, mayor es su efectividad.

**Russell Westbrook**:

- Contrariamente a los otros jugadores, Westbrook es más efectivo en el rango medio (15 a 19 pies).

**Seth Curry**:

- Aunque presenta alta efectividad en el rango cercano (0 a 4 pies), también mantiene una notable capacidad de anotación en el rango medio (15 a 19 pies).

**Chris Paul**:

- Similar a Seth Curry, destaca por su alta efectividad en tiros cercanos al aro (0 a 4 pies).

**Trae Young**:

- Tiene una mayor probabilidad de anotar en el rangos de (0, 4), similar a los demas jugadores.

**Conclusiones Generales**

- **Cercanía al aro**: Para todos los jugadores, la probabilidad de anotar disminuye consistentemente a medida que aumenta la distancia al aro.

**Estrategias individuales**:
- Russell Westbrook: Enfocarse en tiros desde el rango medio, donde es más efectivo.
- Chris Paul y Seth Curry: Optimizar las oportunidades cercanas al aro para maximizar su rendimiento.
- Seth Curry: Considerar incluir tiros desde rangos medios como parte de la estrategia.
- Trae Young: Considerar acercarse mas al aro para que sus probabilidades de anotar aumenten. 


In [99]:
probabilities

,SHOOTER,RANGE,TOTAL_ATTEMPTS,SUCCESS_ATTEMPTS,PROBABILITY
0,Chris Paul,"(0, 4)",16,12,0.7500
1,Chris Paul,"(10, 14)",61,28,0.4590
2,Chris Paul,"(15, 19)",67,30,0.4478
3,Chris Paul,"(20, 24)",16,6,0.3750
4,Chris Paul,"(25, 29)",34,16,0.4706
5,Chris Paul,"(5, 9)",22,13,0.5909
6,Russell Westbrook,"(0, 4)",25,10,0.4000
7,Russell Westbrook,"(10, 14)",6,4,0.6667
8,Russell Westbrook,"(15, 19)",23,8,0.3478
9,Russell Westbrook,"(20, 24)",7,1,0.1429


## Validacion Estadistica

### Nivel de Confianza


Para calcular el intervalo de confianza (CI, por sus siglas en inglés), se utiliza la siguiente fórmula:

$$
CI = p \pm z \cdot e
$$

### Donde:
$$
p = \text{Probabilidad de éxito (probabilidad de anotar)}
$$

$$
z = \text{Valor crítico } z \text{ correspondiente al nivel de confianza seleccionado (por ejemplo, } z = 1.96 \text{ para un nivel de confianza del 95\%)}
$$

$$
n = \text{Número total de intentos de tiro}
$$

$$
e = \sqrt{\frac{p(1-p)}{n}}
$$

### Interpretación:
El intervalo de confianza proporciona un rango en el que, con un determinado nivel de confianza, se espera que esté la verdadera probabilidad de anotar para un jugador.

**Chris Paul**:

Los intervalos de confianza son relativamente estrechos, lo que indica que las estimaciones son bastante precisas.
El rango (0, 4) tiene un intervalo de confianza de (0.538, 0.962), lo que sugiere una alta probabilidad de anotar con alta confianza.

**Russell Westbrook**:

Los intervalos de confianza son más amplios en algunos rangos, lo que indica menos precisión en esas estimaciones.
El rango (20, 24) tiene un intervalo de confianza de (-0.116, 0.402), lo que sugiere una baja probabilidad de anotar con menos confianza.

**Seth Curry**:

Los intervalos de confianza son generalmente estrechos, lo que indica estimaciones precisas.
El rango (15, 19) tiene un intervalo de confianza de (0.515, 0.929), lo que sugiere una alta probabilidad de anotar con alta confianza.

**Trae Young**:

Los intervalos de confianza son moderados, lo que indica una precisión aceptable en las estimaciones.
El rango (25, 29) tiene un intervalo de confianza de (0.222, 0.404), lo que sugiere una baja probabilidad de anotar con moderada confianza.


In [112]:
z = norm.ppf(0.975) # area bajo la curba de 95%
n = nba['SCORE'].count()

probabilities.loc[:, 'e'] = probabilities.apply(
    lambda row: np.sqrt((row['PROBABILITY'] * (1 - row['PROBABILITY']))/row['TOTAL_ATTEMPTS']),
    axis=1
)

probabilities.loc[:, 'LOWER_BOUND'] = probabilities.apply(
    lambda row: round(row['PROBABILITY'] - z * row['e'], 3),
    axis=1
    )

probabilities.loc[:, 'UPPER_BOUND'] = probabilities.apply(
    lambda row: round(row['PROBABILITY'] + z * row['e'], 3),
    axis=1
    )

probabilities.loc[:, 'CI_WIDTH'] = probabilities.apply(
    lambda row: round(row['UPPER_BOUND'] - row['LOWER_BOUND'], 3),
    axis=1
    )

fig = go.Figure()

for shooter in probabilities['SHOOTER'].unique():
    
    shooter_data = probabilities[probabilities['SHOOTER'] == shooter]
    
    fig.add_trace(go.Bar(
        x=shooter_data['RANGE'],
        y=shooter_data['PROBABILITY'],
        name=shooter,
        error_y=dict(
            type='data',
            array=shooter_data['UPPER_BOUND'] - shooter_data['PROBABILITY'],
            arrayminus=shooter_data['PROBABILITY'] - shooter_data['LOWER_BOUND']
        )
    ))

# Añadir títulos y etiquetas
fig.update_layout(
    title='Probabilidad de Anotar por Rango de Distancia',
    xaxis_title='Rango de Distancia',
    yaxis_title='Probabilidad de Anotar',
    barmode='group',
)

# Mostrar la gráfica
fig.show()